In [1]:
import xarray as xr
import numpy as np
from math import pi

# Features

In [2]:
data = xr.open_dataset('/mnt/storageapplications/data/PERIFEREIA_Attikis/p3/02062021_temp_wind.grib', engine='cfgrib')

/home/sg/miniconda3/envs/pygdal/lib/python3.10/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


t2m -> Θερμοκρασία στα 2 m ανά ώρα -> Μέγιστη, ελάχιστη και μέση θερμοκρασία ημέρας

In [4]:
data['max_temp'] = data.t2m.max(dim='time')

In [5]:
data['min_temp'] = data.t2m.min(dim='time')

In [6]:
data['mean_temp'] = data.t2m.mean(dim='time')

u10,v10 ->Συνιστώσες του ανέμου -> max_velocity, direction, dominant direction, max_velocity in the dominant direction, 

In [26]:
def majority(x):
    counts = np.array(np.unique(x, return_counts=True)).T
    return counts[-1][0]
def dom_vel(x):
    return max(x)

In [27]:
#ταχύτητα ανέμου - Πυθαγόρειο
data['wind'] = np.sqrt(data.u10**2+data.v10**2)

In [28]:
#Κατεύθυνση ανέμου
data['dir'] = 180 + (180 / pi) * np.arctan2(data.u10, data.v10)

In [29]:
data['dir_cat'] = (data.dir>0).astype(int) + (data.dir>22.5).astype(int) + (data.dir>67.5).astype(int) + \
(data.dir>112.5).astype(int) + (data.dir>157.5).astype(int) + \
(data.dir>202.5).astype(int) + (data.dir>247.5).astype(int) + \
(data.dir>292.5).astype(int) - 7*(data.dir>337.5).astype(int)

In [30]:
#To be checked
data['dom_dir'] = xr.DataArray(np.apply_along_axis(majority,0,data.dir_cat),dims=['latitude','longitude'])

In [31]:
data

<xarray.Dataset>
Dimensions:          (latitude: 2, longitude: 4, time: 24)
Coordinates:
  * latitude         (latitude) float64 38.27 38.02
  * longitude        (longitude) float64 23.09 23.34 23.59 23.84
    number           int64 0
  * time             (time) datetime64[ns] 2021-06-02 ... 2021-06-02T23:00:00
    step             timedelta64[ns] 00:00:00
    surface          float64 0.0
    valid_time       (time) datetime64[ns] 2021-06-02 ... 2021-06-02T23:00:00
Data variables: (12/14)
    u10              (time, latitude, longitude) float32 1.059 1.072 ... 1.028
    v10              (time, latitude, longitude) float32 -0.8836 ... 0.6386
    t2m              (time, latitude, longitude) float32 286.1 286.0 ... 287.5
    max_temp         (latitude, longitude) float32 296.3 297.4 ... 297.5 298.1
    min_temp         (latitude, longitude) float32 285.6 285.2 ... 287.3 286.8
    mean_temp        (latitude, longitude) float32 291.6 291.8 ... 292.4 292.6
    ...               ...
    dir_cat          (time, latitude, longitude) int64 8 7 7 7 8 8 ... 6 4 5 6 6
    dom_dir          (latitude, longitude) int64 8 8 8 8 8 8 8 8
    masked_velocity  (time, latitude, longitude) float32 1.379 nan ... nan nan
    dom_vel          (latitude, longitude) float32 1.504 2.521 ... 2.797 3.458
    res_max          (latitude, longitude) float32 2.379 2.859 ... 3.859 3.458
    dir_max          (latitude, longitude) float32 2.379 2.859 ... 3.859 3.458
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-12T16:23 GRIB to CDM+CF via cfgrib-0.9.1...

In [32]:
data['masked_velocity'] = data.wind.where(data.dir_cat==data.dom_dir , drop=False)

In [33]:
data['dom_vel'] = data['masked_velocity'].max(dim='time')

In [34]:
data['res_max'] = data['wind'].max(dim='time')

In [35]:
data['dir_max'] = data.wind.where(data.wind==data.res_max,drop=False).max(dim='time')

In [36]:
out = data[['dom_dir','dom_vel','res_max','dir_max','max_temp','min_temp','mean_temp']]

In [37]:
out

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 4)
Coordinates:
  * latitude   (latitude) float64 38.27 38.02
  * longitude  (longitude) float64 23.09 23.34 23.59 23.84
    number     int64 0
    step       timedelta64[ns] 00:00:00
    surface    float64 0.0
Data variables:
    dom_dir    (latitude, longitude) int64 8 8 8 8 8 8 8 8
    dom_vel    (latitude, longitude) float32 1.504 2.521 3.175 ... 2.797 3.458
    res_max    (latitude, longitude) float32 2.379 2.859 3.329 ... 3.859 3.458
    dir_max    (latitude, longitude) float32 2.379 2.859 3.329 ... 3.859 3.458
    max_temp   (latitude, longitude) float32 296.3 297.4 296.9 ... 297.5 298.1
    min_temp   (latitude, longitude) float32 285.6 285.2 285.7 ... 287.3 286.8
    mean_temp  (latitude, longitude) float32 291.6 291.8 291.6 ... 292.4 292.6
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-12T16:23 GRIB to CDM+CF via cfgrib-0.9.1...

# Features to shapefile

In [38]:
import geopandas as gpd

In [39]:
points = gpd.read_file('/mnt/storageapplications/data/PERIFEREIA_Attikis/p3/grid_wgs84.shp')

In [41]:
#Explode because it is multipoint
points = points.explode(index_parts=False)

In [42]:
points

,id,geometry
0,543,POINT (23.39644 38.21008)
1,544,POINT (23.39758 38.21008)
2,545,POINT (23.39872 38.21009)
3,546,POINT (23.39986 38.21010)
4,547,POINT (23.40100 38.21010)
...,...,...
47446,69345,POINT (23.40020 38.02985)
47447,69685,POINT (23.39565 38.02892)
47448,69686,POINT (23.39679 38.02893)
47449,69687,POINT (23.39792 38.02893)


In [21]:
def features(row,feature):
    lon = row['geometry'].x
    lat = row['geometry'].y
    sel = out.sel(latitude= lat,longitude=lon, method="nearest")
    value = sel[feature].data
#     dom_vel = float(sel.dom_vel.data)
#     res_max = float(sel.res_max.data)
#     dir_max = float(sel.dir_max.data)
#     max_temp = float(sel.max_temp.data)
#     min_temp = float(sel.min_temp.data)
#     mean_temp = float(sel.mean_temp.data)
#    return dom_dir,dom_vel,res_max,dir_max,max_temp,min_temp,mean_temp
    return value

In [22]:
new_features = ['dom_dir','dom_vel','res_max','dir_max','max_temp','min_temp','mean_temp']

In [52]:
for feat in new_features:
    points[feat] = points.apply(lambda row: features(row,feat),axis=1)

8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


...

297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658


297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216


297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
297.40216
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838


296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.86658
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838
296.33838



KeyboardInterrupt



In [53]:
points

,id,geometry,dom_dir,dom_vel,res_max,dir_max
0,543,POINT (23.39644 38.21008),8,2.5210607,2.858999,2.858999
1,544,POINT (23.39758 38.21008),8,2.5210607,2.858999,2.858999
2,545,POINT (23.39872 38.21009),8,2.5210607,2.858999,2.858999
3,546,POINT (23.39986 38.21010),8,2.5210607,2.858999,2.858999
4,547,POINT (23.40100 38.21010),8,2.5210607,2.858999,2.858999
...,...,...,...,...,...,...
47446,69345,POINT (23.40020 38.02985),8,2.1285074,4.0351934,4.0351934
47447,69685,POINT (23.39565 38.02892),8,2.1285074,4.0351934,4.0351934
47448,69686,POINT (23.39679 38.02893),8,2.1285074,4.0351934,4.0351934
47449,69687,POINT (23.39792 38.02893),8,2.1285074,4.0351934,4.0351934


In [46]:
point.to_file('/path/for/save/filename.shp')

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 4)
Coordinates:
  * latitude   (latitude) float64 38.27 38.02
  * longitude  (longitude) float64 23.09 23.34 23.59 23.84
    number     int64 0
    step       timedelta64[ns] 00:00:00
    surface    float64 0.0
Data variables:
    dom_dir    (latitude, longitude) int64 8 8 8 8 8 8 8 8
    dom_vel    (latitude, longitude) float32 1.504 2.521 3.175 ... 2.797 3.458
    res_max    (latitude, longitude) float32 2.379 2.859 3.329 ... 3.859 3.458
    dir_max    (latitude, longitude) float32 2.379 2.859 3.329 ... 3.859 3.458
    max_temp   (latitude, longitude) float32 296.3 297.4 296.9 ... 297.5 298.1
    min_temp   (latitude, longitude) float32 285.6 285.2 285.7 ... 287.3 286.8
    mean_temp  (latitude, longitude) float32 291.6 291.8 291.6 ... 292.4 292.6
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-12T16:23 GRIB to CDM+CF via cfgrib-0.9.1...

# Join with static csvs

https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4

In [54]:
import pandas as pd

In [ ]:
dem = pd.read_csv('')